eval

In [1]:
import sys

import pandas as pd
import tensorflow as tf
from preprocessing import clean_text 
from model import build_model, train_model, evaluate_model, predict_emotion
from utils import *
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json
import keras
import pickle
import os

### LOAD

In [2]:
# Load dataset
df = pd.read_csv('text.csv')

# Preprocess data
df = df.drop('Number', axis=1)
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])
# Manually create the label mapping
label_mapping = dict(zip(label_encoder.inverse_transform(df['label_encoded']), df['label_encoded']))
df=df.drop('label',axis=1)
df=df.rename(columns={'label_encoded':'label'})

unique_review = df['text'].unique()


df['cleaned_text'] = df['text'].apply(clean_text)

df['cleaned_text'] = df['cleaned_text'].str.replace("http", "").str.replace("href", "").str.replace("img", "").str.replace("irc", "")

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_text'], df['label'], test_size=0.3, random_state=42)

In [3]:
tokenizer = Tokenizer(num_words=50000)
tokenizer.fit_on_texts(X_train)
X_train_padded = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=100, padding='post')
X_test_padded = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=100, padding='post')
with open('tokenizer.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)
# Load the model architecture and weights
"""json_file2 = open('model.json', 'r')
loaded_model_json2 = json_file2.read()
json_file2.close()
loaded_model2 = model_from_json(loaded_model_json2)
loaded_model2.load_weights("model.weights.h5")"""
mymodel = build_model(input_dim=50000, output_dim=len(label_encoder.classes_))
myhistory = train_model(mymodel, X_train_padded, y_train, X_test_padded, y_test, epochs=50)
model_json = mymodel.to_json()
with open("model2.json", "w") as json_file:
    json_file.write(model_json)
    # serialize weights to HDF5
mymodel.save_weights("model2.weights.h5")

Epoch 1/50
9118/9118 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.8273 - loss: 0.4222 - val_accuracy: 0.9174 - val_loss: 0.1600
Epoch 2/50
9118/9118 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.9238 - loss: 0.1487 - val_accuracy: 0.9177 - val_loss: 0.1587
Epoch 3/50
9118/9118 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.9292 - loss: 0.1321 - val_accuracy: 0.9149 - val_loss: 0.1590
Epoch 4/50
9118/9118 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.9341 - loss: 0.1191 - val_accuracy: 0.9143 - val_loss: 0.1663
Epoch 5/50
9118/9118 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.9374 - loss: 0.1103 - val_accuracy: 0.9113 - val_loss: 0.1718


In [8]:
json_file2 = open('model2.json', 'r')
loaded_model_json2 = json_file2.read()
json_file2.close()
loaded_model2 = model_from_json(loaded_model_json2)
loaded_model2.load_weights("model2.weights.h5")

In [9]:
# Evaluate model
loss, accuracy = evaluate_model(loaded_model2, X_test_padded, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

3908/3908 ━━━━━━━━━━━━━━━━━━━━ 4s 894us/step - accuracy: 0.9179 - loss: 0.1587
Test Loss: 0.15868227183818817
Test Accuracy: 0.9177243113517761
